# Use OOP tooling to generate parameter files
Leverage `ppe_tools` developed by NCAR PPE team

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import xarray as xr

import sys

sys.path.append("/glade/u/home/bbuchovecky/ctsm6_ppe/gen_ensembles/gen_paramfiles/")

from ppe_tools import Ensemble, Member, ParamInfo
from ppe_tools.utils import get_default, parse_val

# 1. Generate parameter files

In [2]:
# Load file with parameter ranges
csv = "coupPPE-hist_OAAT_crosswalk.csv"
params = pd.read_csv(csv)

# Where to save the parameter files
pdir = "/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles"

# Where to save the namelist mods
ndir = "/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/namelist_mods"

# Default parameter file to start with
basefile = f"{pdir}/coupPPE-hist.000.nc"

# instantiate the Ensemble object
x = Ensemble(basefile, pdir, ndir)

In [4]:
# Create dictionary for the independent parameters
oaats = {}
names = params["name"]
flags = params["flag"]
for name, flag in zip(names, flags):
    if not pd.notnull(flag):
        print(name)
        ix = params["name"] == name
        minval = params["min"][ix].values[0]
        maxval = params["max"][ix].values[0]
        pftmin = params["pft_mins"][ix].values[0]
        pftmax = params["pft_maxs"][ix].values[0]
        thisloc = params["loc"][ix].values[0]

        needs_pft = minval == "pft"
        if needs_pft:
            thismin = np.zeros((79))
            pftmin_np = np.fromstring(pftmin, dtype="float", sep=",")
            thismin[0 : len(pftmin_np)] = pftmin_np
            # thismin[0:17] = np.fromstring(pftmin, dtype='float', sep=',')
        elif isinstance(minval, str) and "percent" in minval:
            thismin = minval
        else:
            thismin = np.array(float(minval))

        needs_pft = maxval == "pft"
        if needs_pft:
            thismax = np.zeros((79))
            pftmax_np = np.fromstring(pftmax, dtype="float", sep=",")
            thismax[0 : len(pftmax_np)] = pftmax_np
            # thismax[0:17] = np.fromstring(pftmax, dtype='float', sep=',')
        elif isinstance(minval, str) and "percent" in maxval:
            thismax = maxval
        else:
            thismax = np.array(float(maxval))

        oaats[name] = {"min": thismin, "max": thismax, "loc": thisloc}

fff
jmaxb0
medlynslope
medlynintercept
maximum_leaf_wetted_fraction
kmax
leafcn
d_max
psi50
cv
lmrha
FUN_fracfixers


In [8]:
# Add to the Ensemble object
prefix = "coupPPE-hist."
nextnum = 1
x.add_oaats(oaats, prefix, nextnum, skipBFB=True)

In [13]:
oaats["jmaxb0"]

{'min': array(0.01), 'max': array(0.05), 'loc': 'P'}

In [9]:
# Add params that move in unison to the Ensemble object
flags = params["flag"]
ix = pd.notnull(params["flag"])
uflags = pd.unique(flags[ix])
sgns = {"min": "-", "max": ""}
for uflag in uflags:
    print(uflag)
    names = params["name"][flags == uflag]
    for minmax in ["min", "max"]:
        mf = {}

        for name in names:
            print(" ", name, minmax)
            ix = params["name"] == name

            thisval = params[minmax][ix].values[0]
            pftval = params["pft_" + minmax + "s"][ix].values[0]
            thisloc = params["loc"][ix].values[0]

            needs_pft = thisval == "pft"
            if needs_pft:
                val = np.fromstring(pftval, dtype="float", sep=",")
            elif "percent" in thisval:
                val = sgns[minmax] + thisval
            else:
                val = np.array(float(thisval))

            mf[name] = {"value": val, "loc": thisloc, "minmax": minmax, "flag": uflag}

        x.add_mf(mf, prefix)

ACCLIM_SF
  jmaxse_sf min
  tpuse_sf min
  vcmaxse_sf min
  jmaxse_sf max
  tpuse_sf max
  vcmaxse_sf max
KCN
  akc_active min
  akn_active min
  ekc_active min
  ekn_active min
  kc_nonmyc min
  kn_nonmyc min
  akc_active max
  akn_active max
  ekc_active max
  ekn_active max
  kc_nonmyc max
  kn_nonmyc max


In [7]:
# write the parameter files and namelist mods 
csvfile= 'coupPPE-hist_OAAT.csv'
x.write(oaatfile=csvfile)

# 2. Check parameter files
I manually edited `old/coupPPE-hist_crosswalk.csv` to match `coupPPE-hist_OAAT_crosswalk.csv`

In [41]:
def check_perturbed_param(param):
    """Compares the perturbed parameter file to the default parameter file."""
    np.set_printoptions(precision=4)

    def all_equal(lst):
        return all(x == lst[0] for x in lst)

    pdir = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles'
    crosswalk = pd.read_csv("coupPPE-hist_crosswalk_edited.csv", index_col=0, dtype=str)

    if crosswalk.loc[crosswalk["param"]==param]["loc"].unique().item() == "N":
        print("".ljust(50, "-"))
        print(param, "not located in parameter file")
        print("".ljust(50, "-"))
        return

    paramxc_min = crosswalk.loc[(crosswalk["param"]==param) & (crosswalk["minmax"]=="min")]
    paramxc_max = crosswalk.loc[(crosswalk["param"]==param) & (crosswalk["minmax"]=="max")]

    defpfile = xr.open_dataset(f"{pdir}/coupPPE-hist.000.nc", decode_timedelta=False)
    minpfile = xr.open_dataset(f"{pdir}/coupPPE-hist.{paramxc_min.key.item()}.nc", decode_timedelta=False)
    maxpfile = xr.open_dataset(f"{pdir}/coupPPE-hist.{paramxc_max.key.item()}.nc", decode_timedelta=False)
    prtpfile = {"min": minpfile, "max": maxpfile}

    # Get the component grouped parameters if necessary
    print("".ljust(50, "-"))
    print(param)
    plist = [param]
    if paramxc_min["loc"].item() in ("KCN", "ACCLIM_SF") and paramxc_max["loc"].item() == paramxc_min["loc"].item():
        grouped = paramxc_min["loc"].item()
        plist = list(crosswalk.loc[crosswalk["flag"] == grouped]["param"].unique())

    for p in plist:
        if len(plist) > 1:
            print("".ljust(50, "-"))
            print(">", p)
        
        defp = defpfile[p]

        pdims = [defp.dims]
        pshape = [defp.shape]

        for m in ["min", "max"]:
            print("".ljust(50, "-"))

            pxc = crosswalk.loc[crosswalk["param"]==p].loc[crosswalk["minmax"]==m]
            pert = pxc["pert"]
            pft_pert = pxc["pft_pert"]

            prtp = prtpfile[m][p]
            pdims.append(prtp.dims)
            pshape.append(prtp.shape)

            print(f"{m} | dim: {prtp.dims} | shape: {prtp.shape}")
            
            if pert.values == "pft":
                print("xc pft pert =", np.array(pft_pert.item().split(','), dtype=float))
                print("pft pert    =", prtp.values)
                if not isinstance(pft_pert.item(), str):
                    print("equal       =", np.array_equal(np.array(pft_pert.item().split(','), dtype=float), prtp.values))
            else:
                print("xc pert     =", pert.item())
                print("pert        =", prtp.values)
                if not isinstance(pert.item(), str):
                    print("equal       =", np.array_equal(np.array([pert.item()], dtype=float), np.array([prtp.values], dtype=float)))
        
            print("def         =", defp.values)
            print("pert/def    =", prtp.values / defp.values)

        assert all_equal(pdims), "mismatch dimensions"
        assert all_equal(pshape), "mismatch shapes"

In [42]:
import warnings

pdir = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles'
ndir = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/namelist_mods'
crosswalk = pd.read_csv("coupPPE-hist_crosswalk_edited.csv", index_col=0, dtype=str)
crosswalk_pert = crosswalk.loc[(crosswalk["param"]!="default") & (crosswalk["param"]!="sand_pf")]

for param in crosswalk_pert["param"].unique():
    row = crosswalk_pert.loc[(crosswalk_pert["param"]==param) & (crosswalk_pert["minmax"]=="min")]
    if (row["flag"].item() != "KCN") and (row["flag"].item() != "ACCLIM_SF"):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            check_perturbed_param(row["param"].item())

--------------------------------------------------
fff
--------------------------------------------------
min | dim: () | shape: ()
xc pert     = 0.02
pert        = 0.02
def         = 0.5
pert/def    = 0.04
--------------------------------------------------
max | dim: () | shape: ()
xc pert     = 5
pert        = 5.0
def         = 0.5
pert/def    = 10.0
--------------------------------------------------
jmaxb0
--------------------------------------------------
min | dim: () | shape: ()
xc pert     = 0.01
pert        = 0.01
def         = 0.0311
pert/def    = 0.3215434083601286
--------------------------------------------------
max | dim: () | shape: ()
xc pert     = 0.05
pert        = 0.05
def         = 0.0311
pert/def    = 1.6077170418006432
--------------------------------------------------
medlynslope
--------------------------------------------------
min | dim: ('pft',) | shape: (79,)
xc pft pert = [9.   1.53 1.53 1.53 1.7  1.7  2.05 2.05 2.05 0.91 0.91 0.91 3.78 3.78
 0.65 3.89 3.89

In [45]:
import os
directory = ndir
for fname in sorted(os.listdir(directory)):
    fpath = os.path.join(directory, fname)
    if os.path.isfile(fpath) and fname.endswith('.txt'):
        with open(fpath, 'r') as f:
            content = f.read()
        print(f"{fname[:-4]}", end=" ")
        content = content.split("\n")
        if len(content) > 2:
            print(content[-2], end="")
        print()

coupPPE-hist.000 
coupPPE-hist.001 
coupPPE-hist.002 
coupPPE-hist.003 
coupPPE-hist.004 
coupPPE-hist.005 
coupPPE-hist.006 
coupPPE-hist.007 
coupPPE-hist.008 
coupPPE-hist.009 maximum_leaf_wetted_fraction=0.01
coupPPE-hist.010 maximum_leaf_wetted_fraction=0.5
coupPPE-hist.011 
coupPPE-hist.012 
coupPPE-hist.013 
coupPPE-hist.014 
coupPPE-hist.015 
coupPPE-hist.016 
coupPPE-hist.017 
coupPPE-hist.018 
coupPPE-hist.019 
coupPPE-hist.020 
coupPPE-hist.021 
coupPPE-hist.022 
coupPPE-hist.023 
coupPPE-hist.024 
coupPPE-hist.025 
coupPPE-hist.026 
coupPPE-hist.027 
coupPPE-hist.028 
